## Dev Notebook 2024-11

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import expand_labels
from skimage.measure import label
import skimage
import numpy as np
import os,sys 

In [ ]:
%matplotlib inline

In [ ]:
path = os.path.abspath(os.path.join(os.path.join(os.path.abspath(""), os.pardir), os.pardir))
print(path)
sys.path.insert(1,path)
import neurotorchmz
from neurotorchmz.utils.image import ImgObj
from neurotorchmz.utils.synapse_detection import *
print(neurotorchmz.__version__)
%matplotlib inline

In [ ]:
neurotorchmz.Start_Background(neurotorchmz.Edition.NEUROTORCH_DEBUG)
print(neurotorchmz.__version__)

In [ ]:
imgObj = neurotorchmz.Get_API().ImageObject
diffMax = imgObj.imgDiffView(ImgObj.SPATIAL).Max

plt.imshow(imgObj.imgView(ImgObj.SPATIAL).Mean)
plt.show()
plt.imshow(diffMax)
plt.show()

In [ ]:
image_max = ndi.maximum_filter(diffMax, size=20, mode="constant")
plt.imshow(image_max)

In [ ]:
image_max_th = (image_max > 90) * image_max
plt.imshow(image_max_th)

In [ ]:
maxima_mask = image_max_th == diffMax
maxima = np.where(maxima_mask)
print(maxima)

In [ ]:
labeledImage = label(maxima_mask)
print(labeledImage)

In [ ]:
plt.imshow(diffMax)
plt.scatter(maxima[1], maxima[0], marker="x", c="orange")

In [ ]:
rois: list[ISynapseROI] = []
for i in range(len(maxima[0])):
    x, y = maxima[1][i], maxima[0][i]
    rois.append(CircularSynapseROI().SetLocation(x, y).SetRadius(6))
synapses = SingleframeSynapse.ROIsToSynapses(rois)
neurotorchmz.Get_API().SetDetectionResult(synapses)

In [ ]:
import neurotorchmz.gui
import neurotorchmz.gui.tab3


gui = neurotorchmz.Get_API().gui

tab3: neurotorchmz.gui.tab3.Tab3 = gui.tabs["Tab3"]
tab3.detectionResult.modified = False
tab3.detectionResult.SetISynapses(synapses)
tab3.Invalidate_ROIs()

### Thresholding Tests

In [ ]:
peaks = peak_local_max(diffMax, min_distance=20, threshold_abs=90)
plt.imshow(diffMax)
plt.scatter(peaks[:, 1], peaks[:, 0], marker="x", c="orange")

In [ ]:
imgDiff_th = (diffMax >= 50)
plt.imshow(imgDiff_th)

In [ ]:
expanded = expand_labels(labeledImage, distance=12)
plt.imshow(expanded)

In [ ]:
expanded_th = expanded * (imgDiff_th != 0)
plt.imshow(expanded_th)
plt.show()
regprops = skimage.measure.regionprops(expanded_th, intensity_image=diffMax)
print(regprops)

In [ ]:
plt.imshow(regprops[20].image_filled)

In [ ]:
footprint = np.full(shape=(5,5), fill_value=True)
print(footprint)

local_max = skimage.morphology.local_maxima(diffMax, footprint=footprint)
plt.imshow(local_max)

In [ ]:
threshold = 60
image_th = diffMax >= threshold
image_th_labeled = label(image_th, connectivity=1)
print("TH Labels", np.max(image_th_labeled))

labeled_maxima = label(maxima_mask)
expanded_maxima = expand_labels(labeled_maxima, distance=12)

labels = []
for i in range(len(maxima[0])):
    x, y = maxima[1][i], maxima[0][i]
    label_th = image_th_labeled[y,x]
    labels.append(label_th)

for l in labels:
    image_th_labeled[image_th_labeled == l] = -1

image_final = image_th_labeled == -1
labeled_final = label(image_final)


plt.imshow(image_final)
#plt.scatter(maxima[1], maxima[0], marker="x", c="orange")
plt.show()

In [ ]:
regprops = skimage.measure.regionprops(labeled_final, intensity_image=diffMax)

In [ ]:
regprops[0]

In [ ]:
plt.imshow(np.pad(regprops[0].image, 1, mode="constant", constant_values=0))

In [ ]:
rois: list[ISynapseROI] = []

for r in regprops:
    if r.area > 20:
        y, x = r.centroid_weighted
        x, y = int(x), int(y)
        rois.append(CircularSynapseROI().SetLocation(x, y).SetRadius(6))

synapses = SingleframeSynapse.ROIsToSynapses(rois)
neurotorchmz.Get_API().SetDetectionResult(synapses)

In [ ]:
tab3 = neurotorchmz.Get_API().GUI.tabs["Tab3"]
detect = tab3.detectionAlgorithm

In [ ]:
print(detect.maxima)

In [ ]:
plt.imshow(tab3.detectionAlgorithm.imgMaximumFiltered)
#plt.scatter(tab3.detectionAlgorithm.maxima[1], tab3.detectionAlgorithm.maxima[0], marker="x", c="orange")
plt.scatter(tab3.detectionAlgorithm.maxima[:, 1], tab3.detectionAlgorithm.maxima[:, 0], marker="x", c="orange")


In [ ]:
for region in detect.region_props:
    plt.imshow(region.image_filled)
    plt.show()

In [ ]:
plt.imshow(tab3.detectionAlgorithm.maxima_labeled_expanded)

In [ ]:
plt.imshow(detect.imgThresholded_labeled)

In [ ]:
print(detect._labels)

In [ ]:
plt.imshow(detect.labeledImage)

### ImageJ

In [ ]:
aIP = neurotorchmz.API.GUI.ijH._activeImagePlus

In [ ]:
fI = aIP.getFileInfo()

In [ ]:
ip = aIP.get
ip.

In [ ]:
path = os.path.abspath(os.path.join(os.path.join(os.path.abspath(""), os.pardir), os.pardir))
print(path)
sys.path.insert(1,path)
import neurotorchmz

In [ ]:
for k in sys.modules.keys():
    if "neurotorch" in k:
        print(k)

In [ ]:
neurotorchmz.API.OpenFile(path=r"D:\Eigene Datein\Programmieren\PYTHON\AG Heine Hiwi\Daten\+40118_CS1-2_37a+47_iGlu_denoised.tif", waitCompletion=True)

In [ ]:
neurotorchmz.API.AutoDetect(waitCompletion=True)

In [ ]:
neurotorchmz.API.ExportAsCSV()

In [ ]:
for s in neurotorchmz.API.TabROIFinder_DetectionResult.synapses:
    s: SingleframeSynapse = s
    roi = s.synapse
    plt.imshow(roi.regionProps.image)
    plt.title(roi.LocationStr())
    plt.show()

In [ ]:

import neurotorchmz.gui.tab3

tab3 = neurotorchmz.API.GUI.tabs[neurotorchmz.gui.tab3.TabROIFinder]

In [ ]:
for s in neurotorchmz.API.TabROIFinder_DetectionResult.synapses:
    s: SingleframeSynapse = s
    roi = s.synapse
    print(np.min(roi.regionProps.image_intensity), np.max(roi.regionProps.image_intensity))
    plt.imshow(roi.regionProps.image_intensity)
    plt.title(roi.LocationStr())
    plt.show()

In [ ]:
for x in tab3.detectionAlgorithm._imgSynapse:
    print(x)
    plt.imshow(x)
    plt.show()

In [ ]:
from skimage.draw import disk
from matplotlib.patches import Circle

In [ ]:
for radius in range(10):
    shape = (10,10)
    _img = np.zeros(shape)
    point = (5,5)
    xmax = _img.shape[1]
    ymax = _img.shape[0]
    for x in range(point[0]-radius,point[0]+2*radius+1):
        for y in range(point[1]-radius,point[1]+2*radius+1):
            if ((x-point[0])**2+(y-point[1])**2)<radius**2+2**(1/2) and x >= 0 and y >= 0 and x < xmax and y < ymax:
                _img[x,y] = 1
    _img2 = np.zeros(shape, dtype=np.uint8)
    _img2[disk(point, radius+0.4999, shape=shape)] = 1

    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(_img)
    axes[1].imshow(_img2)
    c = Circle(point, radius+0.5, fill=False, color="red")
    c2 = Circle(point, radius+0.5, fill=False, color="red")
    axes[0].add_patch(c)
    axes[1].add_patch(c2)
    axes[0].grid(color='w', linestyle='-', linewidth=1)
    axes[1].grid(color='w', linestyle='-', linewidth=1)
    axes[0].set_xticks(np.arange(0, shape[0], 1))
    axes[0].set_yticks(np.arange(0, shape[1], 1))
    axes[1].set_xticks(np.arange(0, shape[0], 1))
    axes[1].set_yticks(np.arange(0, shape[1], 1))
    fig.show()

In [ ]:
shape = (20, 20)
img = np.zeros(shape, dtype=np.uint8)
img[disk((10,10), 6, shape=shape)] = 1
plt.imshow(img)
plt.show()

In [ ]:
imgObj = neurotorchmz.API.ImageObject

In [ ]:
signal = imgObj.imgDiff[self.GetImageMask(imgObj.imgDiff.shape)]

In [ ]:
a = [1,2,3,4,5]
print(a[-3:])